In [9]:
import requests
import pandas as pd

# Tu token de Banxico
token = "b2b8aef3c6559490348dbc3e9bfbe878054a181310b39ac2e33793ac6186b216"

# Serie del tipo de cambio FIX (peso/dólar)
serie_id = "SF43718"

# Endpoint
url = f"https://www.banxico.org.mx/SieAPIRest/service/v1/series/{serie_id}/datos"

# Headers con el token
headers = {"Bmx-Token": token}

# Llamada a la API
response = requests.get(url, headers=headers)
data = response.json()

# Extraer datos
datos = data["bmx"]["series"][0]["datos"]

# Convertir a DataFrame
df = pd.DataFrame(datos)
df["fecha"] = pd.to_datetime(df["fecha"], dayfirst=True)
df["dato"] = pd.to_numeric(df["dato"], errors="coerce")

df.head()


,fecha,dato
0,1991-11-12,3.0735
1,1991-11-13,3.0712
2,1991-11-14,3.0718
3,1991-11-15,3.0684
4,1991-11-18,3.0673
